In [1]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer
import pandas as pd

import torch

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [3]:
model = AutoModelForCausalLM.from_pretrained('/home/uj-user/Yo/hybrid-ltm/model/llama-7b_MT_DG', 
                                             torch_dtype=torch.bfloat16,
                                            #  device_map="auto",
                                            #  max_memory=[23, 23, 23, 23],
                                             load_in_8bit=False)
model = model.to(device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
tokenizer = LlamaTokenizer.from_pretrained('/home/uj-user/Yo/hybrid-ltm/model/llama-7b_MT_DG', use_fast=False)

# from transformers import LlamaTokenizer
# tokenizer = LlamaTokenizer.from_pretrained('daryl149/llama-2-7b-hf', truncation_side='left')
# tokenizer.add_special_tokens({'pad_token': '</s>'})
# tokenizer.eos_token = tokenizer.pad_token 
tokenizer

LlamaTokenizer(name_or_path='/home/uj-user/Yo/hybrid-ltm/model/llama-7b_MT_DG', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='left', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
data = load_dataset('/home/uj-user/Yo/hybrid-ltm/data/tasks/MT_DG')
data

Found cached dataset json (/home/uj-user/.cache/huggingface/datasets/json/MT_DG-e1e4da65bf530ac9/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['input', 'output'],
        num_rows: 1000
    })
})

In [6]:
def tokenize_func(example):
    return tokenizer(example['input'], truncation=True, max_length=2048, return_tensors='pt', add_special_tokens=False)

tokenized_dataset = data.map(tokenize_func, num_proc=96)

Map (num_proc=96):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=96):   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=96):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
q_pre = "<s>\n"
qa_link = "\n"
TarLen=256
MaxLen = 2048

In [8]:
input_ids = tokenized_dataset['validation']['input_ids'][:2]

In [9]:
model = model.eval()
for input in data['validation']['input']:
    
    input = q_pre + input +qa_link
    input_ids = tokenizer([input], truncation=True, max_length=(MaxLen - TarLen), add_special_tokens=False).input_ids
    target_len = min(len(input_ids[0])+TarLen, MaxLen)

    output_ids = model.generate(
    torch.as_tensor(input_ids).to(device),
    do_sample=True,
    temperature=0.2,
    max_length=target_len,
)
    # print(input)
    output_ids = output_ids[0][len(input_ids[0]):]
    print(tokenizer.decode(output_ids, skip_special_tokens=True).strip())
    break

Co-workers A: Actually, I was thinking about the time when you made a big mistake and I didn't ask for help. I think I was just too nervous to ask for help.


In [10]:
output_ids

tensor([ 7967, 29899,  1287,   414,   319, 29901, 12823, 29892,   306,   471,
         7291,  1048,   278,   931,   746,   366,  1754,   263,  4802, 10171,
          322,   306,  3282, 29915, 29873,  2244,   363,  1371, 29889,   306,
         1348,   306,   471,   925,  2086, 23547,   681,   304,  2244,   363,
         1371, 29889,    13,    13,     2], device='cuda:0')

In [11]:
print(tokenizer.decode(output_ids).strip())

Co-workers A: Actually, I was thinking about the time when you made a big mistake and I didn't ask for help. I think I was just too nervous to ask for help.

</s>


: 